In [1]:
### How to save results for visual inspection

In [2]:
from utils import visualization

In [3]:
visualization.save_models_results(["2D_Unet_standard"], "ACDC_training")

Constructing model:  2D_Unet_standard
Number of classes:  4
Loading model from:  experiments/ACDC_training/2D_Unet_standard/model_checkpoint.pt
Weights loaded successfully!
Total number of parameters of model:  4320132
Total number of trainable parameters of model:  4320132 

Using per slice normalization!


In [4]:
# to save double seg predictions

import torch

import general_config
import constants
import numpy as np
import save_complete_nii_results
from utils import inference
from utils import prepare_models
from utils.training_utils.box_utils import convert_offsets_to_bboxes, wh2corners
from utils.training_utils import training_processing
from utils.ROI_crop import roi_crop
from utils.ROI_crop import predict_ROI
from utils.training_utils import prints, training_setup
from utils import metrics

def get_double_seg_predictions(model_a, model_b, dataset, logsoft_preds=False):
    """
    Args:
    model_a: str -> model to extract ROI (standard)
    model_b: str -> model to predict from ROI
    dataset: str
    """
    models, params_list, configs_list = prepare_models.prepare([model_a, model_b], dataset)
    model_a, model_b = models
    validation_loader = training_setup.prepare_val_loader(params_list[0], configs_list[0])
    validation_loader_2 = training_setup.prepare_val_loader(params_list[1], configs_list[1])
    
    coords_n_scores = predict_ROI.get_segmentor_pred_coords(validation_loader, model_a, params_list[0], configs_list[0])
    _, predictions_double_seg = predict_ROI.segment_with_computed_ROI(validation_loader_2, model_b, params_list[1], configs_list[1], coords_n_scores, logsoft_preds=logsoft_preds)
    
    return predictions_double_seg, validation_loader_2

In [5]:
double_preds, loader = get_double_seg_predictions("DeepLab_Standard", "2D_Unet_ROI", "ACDC_training")

Constructing model:  DeepLab_Standard
Number of classes:  4
Loading model from:  experiments/ACDC_training/DeepLab_Standard/model_checkpoint.pt
Weights loaded successfully!
Total number of parameters of model:  2412094
Total number of trainable parameters of model:  2412094 

Constructing model:  2D_Unet_ROI
Number of classes:  4
Loading model from:  experiments/ACDC_training/2D_Unet_ROI/model_checkpoint.pt
Weights loaded successfully!
Total number of parameters of model:  4320132
Total number of trainable parameters of model:  4320132 

Dataset mean and std:  69.52275548950034 90.27520888722917
Using per slice normalization!
current volume dice:  [0.92374806 0.91100657 0.97581802] 0.9368575482996508
current volume dice:  [0.82859669 0.91844213 0.97091742] 0.9059854159142487
current volume dice:  [0.94790761 0.89493365 0.97688765] 0.9399096356886073
current volume dice:  [0.95597562 0.89736374 0.97719388] 0.943511078843866
current volume dice:  [0.95727067 0.88903437 0.97462758] 0.9403

In [6]:
visualization.save_predictions(loader, model_names=["DoubleSeg"], dataset_name="ACDC_training", model_pred_lists=[double_preds])

In [7]:
# saving ensemble results is similar

In [8]:
# load the model weigths, again using the previously trained example models
models, params_list, configs_list = prepare_models.prepare(["Deeplab_standard", "2D_Unet_standard"], "ACDC_training")
model_a, model_b = models
validation_loader_a = training_setup.prepare_val_loader(params_list[0], configs_list[0])
validation_loader_b = training_setup.prepare_val_loader(params_list[1], configs_list[1])

predictions_deeplab, _ = inference.run_model_inference(validation_loader_a, model_a, params_list[0], logsoft_preds=True)
predictions_unet, _ = inference.run_model_inference(validation_loader_b, model_b, params_list[1], logsoft_preds=True)

Constructing model:  Deeplab_standard
Number of classes:  4
Loading model from:  experiments/ACDC_training/Deeplab_standard/model_checkpoint.pt
Weights loaded successfully!
Total number of parameters of model:  2412094
Total number of trainable parameters of model:  2412094 

Constructing model:  2D_Unet_standard
Number of classes:  4
Loading model from:  experiments/ACDC_training/2D_Unet_standard/model_checkpoint.pt
Weights loaded successfully!
Total number of parameters of model:  4320132
Total number of trainable parameters of model:  4320132 

Dataset mean and std:  69.52275548950034 90.27520888722917
Using per slice normalization!


In [9]:
# finally, ensemble standard models
logsoftmax_volumes = [predictions_unet, predictions_deeplab]
combined_volumes = inference.ensemble_inference(logsoftmax_volumes, validation_loader_a)

Current volume dice:  [0.92808829 0.89870053 0.97159266]
Current volume dice:  [0.83046192 0.90678258 0.9656294 ]
Current volume dice:  [0.94288436 0.88820693 0.97793967]
Current volume dice:  [0.95561358 0.8818305  0.97560076]
Current volume dice:  [0.96005954 0.89618109 0.97802618]
Current volume dice:  [0.87607362 0.86478476 0.95738979]
Current volume dice:  [0.90919731 0.88735279 0.97329089]
Current volume dice:  [0.94191432 0.87881286 0.95765583]
Current volume dice:  [0.80059645 0.89832744 0.95997748]
Current volume dice:  [0.84998072 0.90283055 0.85839987]
Current volume dice:  [0.93429014 0.92814325 0.96487183]
Current volume dice:  [0.68306802 0.9432475  0.84878049]
Current volume dice:  [0.96439053 0.90982125 0.96103896]
Current volume dice:  [0.90268849 0.92802678 0.90187562]
Current volume dice:  [0.94731396 0.94384664 0.96837974]
Current volume dice:  [0.86651835 0.94617099 0.94200351]
Current volume dice:  [0.93864829 0.88459942 0.97503543]
Current volume dice:  [0.914194

In [12]:
visualization.save_predictions(loader, model_names=["Ensemble"], dataset_name="ACDC_training", model_pred_lists=[combined_volumes])